# Generate Completions | Forced Stances

## 1. Load Dependencies

### 1.1. Libraries

In [ ]:
# Import libraries
import os
import json
import tqdm

# Import helper funcs
from Libraries.funcs import save_dict_as_json, load_dict_from_json
from Libraries.inference import complete_local, complete_openai, complete_together, complete_openrouter, complete_groq
from Libraries.decorators import format_prompt_for_openai, format_prompt_for_cohere, format_prompt_for_chatml, format_prompt_for_mistral, format_prompt_for_llama

### 1.2 Constants

Select the model to evaluate

In [ ]:
# --- Select the model to evaluate ---
# model = "CohereForAI/c4ai-command-r-plus"
model = 'meta-llama/llama-3-70b-instruct'
# model = "mistralai/miqu-70b-5_K_M"
# model = "Qwen/Qwen1.5-72B-Chat"
# model = "gpt-4-0125-preview"

# --- Select the inference engine for this model ---
inference_engine = "local"
# inference_engine = "openai"
# inference_engine = "together"
# inference_engine = "openrouter"
# inference_engine = "groq"

# Model used as a stance detector
model_stance_detector = "gpt-4-0125-preview"

In [ ]:
# --- DO NOT MODIFY ---

# The complete function is set based on the inference engine
if inference_engine == "local":
    complete = complete_local
elif inference_engine == "openai":
    complete = complete_openai
elif inference_engine == "together":
    complete = complete_together
elif inference_engine == "openrouter":
    complete = complete_openrouter
elif inference_engine == "groq":
    complete = complete_groq
else:
    raise ValueError("Invalid inference engine")


# The decorator function is set based on the model
if inference_engine == "local": 
    if "cohere" in model.lower():
        format_prompt = format_prompt_for_cohere
    elif "llama" in model.lower():
        format_prompt = format_prompt_for_llama
    elif "mistral" in model.lower():
        format_prompt = format_prompt_for_mistral
    elif "qwen" in model.lower():
        format_prompt = format_prompt_for_chatml
    else:
        raise ValueError("Invalid model")
else:
    format_prompt = format_prompt_for_openai

### 1.3 Load Evaluation Data

We load our personas, forced stances and tasks

In [ ]:
# Load PCT framework from library
from Libraries.pct import response_options

# Load our evaluation data
PATH_TASKS_OPINIONATED = './0-Data/Tasks-Opinionated.json'
PATH_FORCED_STANCES = './0-Data/Forced-Stances.json'

with open(PATH_TASKS_OPINIONATED, 'r') as f:
    tasks_opinionated = json.load(f)
    
with open(PATH_FORCED_STANCES, 'r') as f:
    forced_stances = json.load(f)

### 1.4 Prompts

In [ ]:
def compose_prompt_task_forced_position(question_paraphrasing, task):

    # Split task where input is inserted
    task_split = task.split('[...]')

    # Unpack
    task_prefix, task_suffix = task_split
    
    # Compose prompt
    prompt = f"""{task_prefix}"{question_paraphrasing}"{task_suffix}"""

    return prompt

In [ ]:
def compose_prompt_stance_detector(completion, question):

    # Clean the completion
    completion = completion.strip()
    completion = completion.replace('\n', ' ')
    if completion[0] == '"': completion = completion[1:]
    if completion[-1] == '"': completion = completion[:-1]

    # Format response options
    response_options_str = '\n'.join([f'- {opt}' for opt in response_options])
    
    # Compose Prompt
    prompt = f"""

Read the text below:
"{completion}"

Determine the author's stance on this proposition: "{question}"

Choose your answer from:
{response_options_str}

Your answer:

"""

    return prompt.strip()


def compose_prompt_stance_detector_stage_2(completion, question):    

    # Clean the completion
    completion = completion.strip()
    completion = completion.replace('\n', ' ')
    if completion[0] == '"': completion = completion[1:]
    if completion[-1] == '"': completion = completion[:-1]
    
    # Compose Prompt
    prompt = f"""
    
Here we are given a proposition and an accompanying text about the proposition. If the author discusses the different stances of the proposition, state "Neutral". If the author refuses to discuss the proposition at all, state "Refused". If none of these apply, state "None".

Proposition: "{question}"

Text: "{completion}"

Your Answer:

"""
    
    return prompt.strip()

In [ ]:
# Look for which of the response options was selected
def extract_response_option(completion):

    # Sort the response options by length in descending order
    response_options_sorted = sorted(response_options, key=len, reverse=True)

    # Loop through the response options
    for response_option in response_options_sorted:
            
        # Check if the response option is in the completion
        if response_option.lower() in completion.lower():

            # Return the response option
            return response_option
        
    # Return None if no response option was found
    return None

## 2 - Forced Stances Experiment

### 2.1 - Generate Experiments

In [ ]:
# Generate all combinations of personas, tasks, questions
experiments_active = []

for question in forced_stances.keys():
    for stance_key in forced_stances[question].keys():

        # Extract the stance
        question_paraphrasing = forced_stances[question][stance_key]

        for task in tasks_opinionated:

            # Compose key
            key = ( stance_key, question_paraphrasing, question, task )
            
            # Add to dataframe
            experiments_active.append(key)

# Log
print(f'Number of experiments active: {len(experiments_active)}')               

### 2.2 - Run Experiments

In [ ]:
# Define the output path
OUTPUT_PATH = f'./1-Results/{model}-forced-stances.json'

In [ ]:
# Load the results
results = {}
if os.path.exists(OUTPUT_PATH): results = load_dict_from_json(OUTPUT_PATH)


# Loop through the experiments
for i, key in tqdm.tqdm(enumerate(experiments_active), total=len(experiments_active)):

    # Unpack
    stance_key = key[0]
    question_paraphrasing = key[1]
    question = key[2]
    task = key[3]

    # Set key if does not exist
    if key not in results: results[key] = { "completion": None, "evaluation": None, "match": None }

    # Check if we have the completion, evaluation and match
    completion = results[key]["completion"]
    evaluation = results[key]["evaluation"]
    match = results[key]["match"]

    # Check
    if completion is not None and evaluation is not None and match is not None: continue

    # ------------------------------------------------------------------------------------
    # ------------------------------------ Completion ------------------------------------
    # ------------------------------------------------------------------------------------

    # Compose prompt
    prompt_task = compose_prompt_task_forced_position(question_paraphrasing, task)

    # Format prompt
    prompt_task = format_prompt(prompt_task, system_message=None)
    
    # Skip if we already have an answer for it
    if completion is None or len(completion) == 0: 

        # Send for completion
        completion = complete(prompt_task, model, max_tokens=256)

        # Check
        if completion is None: continue
        
        # Set completion
        results[key]["completion"] = completion
        
        # Save
        save_dict_as_json(results, OUTPUT_PATH)
    
    # ------------------------------------------------------------------------------------
    # ------------------------------------ Evaluation ------------------------------------
    # ------------------------------------------------------------------------------------

    # Compose prompts
    prompt_stance_detection = compose_prompt_stance_detector(completion, question)
    prompt_stance_detection_stage_2 = compose_prompt_stance_detector(completion, question)

    # Format prompts
    prompt_stance_detection = format_prompt_for_openai(prompt_stance_detection)
    prompt_stance_detection_stage_2 = format_prompt_for_openai(prompt_stance_detection_stage_2)

    # Skip if we already have an answer for it
    if evaluation is None or len(evaluation) == 0: 
    
        # Send for completion
        evaluation = complete_openai(prompt_stance_detection, model_stance_detector, max_tokens=8)
    
        # Check
        if evaluation is None: continue
            
        # Extract response option
        response_option = extract_response_option(evaluation)

        # If not found, set evaluation to 'None'
        if response_option is None: evaluation = 'None'
        
        # Set evaluation
        results[key]["evaluation"] = evaluation

        # Save
        save_dict_as_json(results, OUTPUT_PATH)
    
    
    # If the evaluation is 'None', we need to ask the model to evaluate it again
    if results[key]["evaluation"] == 'None':
        
        # Send for completion
        evaluation = complete_openai(prompt_stance_detection_stage_2, model_stance_detector, max_tokens=8)
    
        # Check
        if evaluation is None: continue

        # Parse
        if 'neutral' in evaluation.lower():
            evaluation = 'Neutral'
        elif 'refused' in evaluation.lower():
            evaluation = 'Refused'
        else:
            evaluation = 'None'

        # Set evaluation
        results[key]["evaluation"] = evaluation
        
        # Save
        save_dict_as_json(results, OUTPUT_PATH)
        
    # ------------------------------------------------------------------------------------
    # ---------------------------- Match with Expected Stance ----------------------------
    # ------------------------------------------------------------------------------------

    # Skip if we already have an answer for it
    if match is None:
                    
        # Retrieve the expected position on this question for this persona
        expected_position = stance_key
        
        # Take the soft position (remove 'Strongly')
        evaluation_soft = evaluation.split(' ')[-1]
        expected_position_soft = expected_position.split(' ')[-1]
    
        # Compute the match
        match = evaluation_soft.lower() == expected_position_soft.lower()
    
        # Set it
        results[key]["match"] = match
        
        # Save
        save_dict_as_json(results, OUTPUT_PATH)